<a href="https://colab.research.google.com/github/jhenningsen/Equity_Analysis/blob/main/StockTradingRange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import time

## `calculate_high_low_range()`

**Purpose:** Calculates the average high-to-low trading range percentage for a single stock over specified periods.

**Parameters:**
- `ticker_symbol` (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT')
- `periods` (list): Number of trading sessions to analyze (default: [5, 10])

**Returns:**
- `dict`: Dictionary containing the ticker and average range percentages for each period
- `None`: If data cannot be retrieved or an error occurs

**How it works:**
1. Downloads the last 30 days of stock data from Yahoo Finance
2. Calculates daily high-to-low range as: `((High - Low) / Low) × 100`
3. Computes the average range percentage for each specified period
4. Returns results in format: `{'Ticker': 'AAPL', '5d_Avg_Range_%': 2.34, '10d_Avg_Range_%': 2.67}`


In [6]:
def calculate_high_low_range(ticker_symbol, periods=[5, 10]):
    """
    Calculate the average high-to-low range in percent for a stock
    over specified trading sessions.

    Parameters:
    ticker_symbol (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT')
    periods (list): List of periods to calculate (default: [5, 10])

    Returns:
    dict: Dictionary with ticker and average ranges for each period
    """

    # Download stock data (get more days to ensure we have enough trading days)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30)

    print(f"Fetching data for {ticker_symbol}...")

    try:
        stock_data = yf.download(ticker_symbol, start=start_date, end=end_date, progress=False)

        if stock_data.empty:
            print(f"  ⚠️ No data found for {ticker_symbol}")
            return None

        # Calculate daily high-to-low range as a percentage
        stock_data['Range_Percent'] = ((stock_data['High'] - stock_data['Low']) / stock_data['Low']) * 100

        # Calculate average ranges for each period
        results = {'Ticker': ticker_symbol}
        for period in periods:
            if len(stock_data) >= period:
                avg_range = stock_data['Range_Percent'].tail(period).mean()
                results[f'{period}d_Avg_Range_%'] = round(avg_range, 2)
            else:
                results[f'{period}d_Avg_Range_%'] = None

        print(f"  ✓ {ticker_symbol} processed successfully")
        return results

    except Exception as e:
        print(f"  ⚠️ Error processing {ticker_symbol}: {str(e)}")
        return None


## `analyze_multiple_stocks()`

**Purpose:** Analyzes multiple stocks and returns a DataFrame with average high-to-low trading range percentages for each ticker.

**Parameters:**
- `ticker_list` (list): List of stock ticker symbols (e.g., ['AAPL', 'MSFT', 'GOOGL'])
- `periods` (list): Number of trading sessions to analyze (default: [5, 10])

**Returns:**
- `pd.DataFrame`: DataFrame with columns for Ticker and average range percentages for each period
- Empty DataFrame if no valid data is retrieved

**How it works:**
1. Iterates through each ticker in the list
2. Calls `calculate_high_low_range()` for each ticker
3. Pauses for 1 second between requests to avoid Yahoo Finance rate limiting
4. Compiles all results into a pandas DataFrame


In [7]:
def analyze_multiple_stocks(ticker_list, periods=[5, 10]):
    """
    Analyze multiple stocks and return a DataFrame with results.

    Parameters:
    ticker_list (list): List of stock ticker symbols
    periods (list): List of periods to calculate (default: [5, 10])

    Returns:
    pd.DataFrame: DataFrame with ticker and average ranges
    """

    print("=" * 60)
    print(f"Stock High-Low Range Analysis for Multiple Stocks")
    print("=" * 60)
    print()

    results_list = []

    for i, ticker in enumerate(ticker_list):
        result = calculate_high_low_range(ticker, periods=periods)
        if result:
            results_list.append(result)

        # Pause for 2 second between requests (except after the last ticker)
        if i < len(ticker_list) - 1:
            time.sleep(2)

    # Create DataFrame
    if results_list:
        df = pd.DataFrame(results_list)
        return df
    else:
        print("\nNo valid data retrieved for any tickers.")
        return pd.DataFrame()


## `__main__` block

**Purpose:** Demonstrates how to use the stock range analysis functions with example tickers and displays formatted results.

**What it does:**
1. Defines a list of example stock tickers (AAPL, MSFT, GOOGL, TSLA, AMZN, NVDA)
2. Calls `analyze_multiple_stocks()` to retrieve trading range data for all tickers
3. Displays the results DataFrame in two formats:
   - Original order as retrieved
   - Sorted by 5-day average range (highest to lowest volatility)

**When it runs:**
- Only executes when the script is run directly (not when imported as a module)


In [8]:
if __name__ == "__main__":
    # Define your list of stock tickers
    tickers = ["TSLA",
               "SPY",
               "NVDA",
               "COIN",
               "QQQ",
               "META",
               "PLTR",
               "AMD",
               "AMZN",
               "MSTR",
               "AAPL",
               "MSFT",
               "GOOGL",
               "LLY",
               "IWM",
               "MU",
               "HOOD",
               "IBIT",
               "SMH",
               "IREN",
               "GLD",
               "NFLX",
               "LULU",
               "APP",
               "AVGO",
               "ORCL"
               ]

    # Get the analysis
    results_df = analyze_multiple_stocks(tickers, periods=[5, 10])

    # Display the results
    if not results_df.empty:
        print("\n" + "=" * 60)
        print("Results Summary:")
        print("=" * 60)
        print()
        print(results_df.to_string(index=False))

        # Optional: Sort by 5-day average range
        print("\n" + "=" * 60)
        print("Sorted by 5-Day Average Range (Highest to Lowest):")
        print("=" * 60)
        print()
        sorted_df = results_df.sort_values(by='5d_Avg_Range_%', ascending=False)
        print(sorted_df.to_string(index=False))

Stock High-Low Range Analysis for Multiple Stocks

Fetching data for TSLA...


/tmp/ipython-input-3631437010.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker_symbol, start=start_date, end=end_date, progress=False)


  ✓ TSLA processed successfully
Fetching data for SPY...


/tmp/ipython-input-3631437010.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker_symbol, start=start_date, end=end_date, progress=False)


  ✓ SPY processed successfully
Fetching data for NVDA...


...

  ✓ APP processed successfully
Fetching data for AVGO...


/tmp/ipython-input-3631437010.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker_symbol, start=start_date, end=end_date, progress=False)


  ✓ AVGO processed successfully
Fetching data for ORCL...
  ✓ ORCL processed successfully

Results Summary:

Ticker  5d_Avg_Range_%  10d_Avg_Range_%
  TSLA            4.15             4.55
   SPY            0.88             0.77
  NVDA            3.08             3.07
  COIN            5.69             5.15
   QQQ            1.25             1.06
  META            3.59             2.56
  PLTR            3.87             3.53
   AMD            4.35             4.07
  AMZN            2.56             2.16
  MSTR            4.87             4.18
  AAPL            2.02             1.70
  MSFT            2.08             1.75
 GOOGL            2.90             2.49
   LLY            4.52             3.39
   IWM            1.44             1.43
    MU            4.63             4.44
  HOOD            5.48             4.70
  IBIT            3.00             2.67
   SMH            2.27             1.98
  IREN           11.68            10.16
   GLD            1.16             1.34
  NFLX     

/tmp/ipython-input-3631437010.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker_symbol, start=start_date, end=end_date, progress=False)


Ticker  5d_Avg_Range_%  10d_Avg_Range_%
  IREN           11.68            10.16
  COIN            5.69             5.15
  HOOD            5.48             4.70
  MSTR            4.87             4.18
   APP            4.64             4.35
    MU            4.63             4.44
   LLY            4.52             3.39
   AMD            4.35             4.07
  AVGO            4.21             3.39
  TSLA            4.15             4.55
  PLTR            3.87             3.53
  ORCL            3.83             3.30
  META            3.59             2.56
  LULU            3.58             3.41
  NVDA            3.08             3.07
  IBIT            3.00             2.67
 GOOGL            2.90             2.49
  NFLX            2.86             2.34
  AMZN            2.56             2.16
   SMH            2.27             1.98
  MSFT            2.08             1.75
  AAPL            2.02             1.70
   IWM            1.44             1.43
   QQQ            1.25             1.06
